In [1]:
sys.path.insert(0, '/home/shengq2/program/spcount/src')

import pandas as pd
import numpy as np
#from spcount.common_util import readFileMap

In [7]:
treeFile="/scratch/vickers_lab/projects/20220707_4893_2_RA_smRNA_mouse_v5_forSpcount/nonhost_genome/refseq_bacteria_table/result/RA_4893_2.species.estimated.count"
outputPrefix="/scratch/vickers_lab/projects/20220707_4893_2_RA_smRNA_mouse_v5_forSpcount/test/RA_4893_2.species"

In [3]:
name_map = {}
with open("/data/cqs/references/spcount/taxonomy.tab", "rt") as fin:
  for line in fin:
    parts = line.rstrip().split('\t')
    name_map[parts[4]] = {
      "id":int(parts[0]),
      "rank_level": int(parts[1]),
      "parent_id": int(parts[2]),
      "rank": parts[3],
      "name": parts[4]
    }

In [5]:
tree_data = pd.read_csv(treeFile, sep="\t")
samples = tree_data.columns[2:]
print(samples)

Index(['IgAKO_SH_2', 'IgAKO_SH_3', 'IgAKO_SH_4', 'IgAKO_SH_5', 'WT_SH_1',
       'WT_SH_2', 'WT_SH_3', 'WT_SH_4', 'WT_SH_5'],
      dtype='object')


In [6]:
find_names = tree_data.Feature.isin(name_map.keys())
np.unique(find_names, return_counts=True)

(array([ True]), array([14838]))

In [21]:

def get_taxonomy_id(name_map, row):  
  return (name_map[row['Feature']]['id'])

def get_rank(name_map, row):  
  return (name_map[row['Feature']]['rank'])

tree_data.insert(loc=1, column="TaxonomyId", value=tree_data.apply(lambda row: get_taxonomy_id(name_map, row), axis=1))
tree_data.insert(loc=2, column="Rank", value=tree_data.apply(lambda row: get_rank(name_map, row), axis=1))
tree_data.head()


,Feature,TaxonomyId,Rank,IgAKO_SH_1,IgAKO_SH_2,IgAKO_SH_3,IgAKO_SH_4,IgAKO_SH_5,WT_SH_1,WT_SH_2,WT_SH_3,WT_SH_4,WT_SH_5
0,Pseudomonas yamanorum,515393,species,10608.96,13471.89,7865.94,9011.63,7384.00,18934.46,5778.83,3730.68,2696.19,8907.07
1,Pseudomonas synxantha,47883,species,3018.41,3450.07,1930.61,2010.35,1659.79,4861.14,1232.17,1018.76,574.15,1760.33
2,Pseudomonas costantinii,168469,species,2753.68,3276.21,1523.98,1660.54,1525.22,4294.71,1011.63,759.45,509.02,1618.72
3,Pseudomonas salomonii,191391,species,21099.40,18135.99,7366.41,9376.93,5992.61,32135.57,6543.74,4718.66,3304.05,8126.20
4,Pseudomonas cedrina,651740,species,2817.78,3307.95,1529.03,1793.95,1741.42,4278.76,1125.90,950.50,609.02,1836.25


In [22]:
new_tree_file=outputPrefix + ".tree.count"
tree_data.to_csv(new_tree_file, sep="\t", index=False)

In [24]:
groupFile="/scratch/vickers_lab/projects/20220707_4893_2_RA_smRNA_mouse_v5_forSpcount/data_visualization/refseq_bacteria_krona_estimated/result/RA_4893_2__fileList1.list"
groups_df=pd.read_csv(groupFile, sep="\t", header=None)
groups_df.rename(columns={groups_df.columns[0]:"sample_name", groups_df.columns.values[1]:"group_name"}, inplace=True)
groups_df.head()

,sample_name,group_name
0,IgAKO_SH_1,IgAKO_SH
1,IgAKO_SH_2,IgAKO_SH
2,IgAKO_SH_3,IgAKO_SH
3,IgAKO_SH_4,IgAKO_SH
4,IgAKO_SH_5,IgAKO_SH


In [25]:
groups_df.loc[len(groups_df.index)] = ['dummy_sample', 'IgAKO_SH'] 
print(groups_df)


     sample_name group_name
0     IgAKO_SH_1   IgAKO_SH
1     IgAKO_SH_2   IgAKO_SH
2     IgAKO_SH_3   IgAKO_SH
3     IgAKO_SH_4   IgAKO_SH
4     IgAKO_SH_5   IgAKO_SH
5        WT_SH_1      WT_SH
6        WT_SH_2      WT_SH
7        WT_SH_3      WT_SH
8        WT_SH_4      WT_SH
9        WT_SH_5      WT_SH
10  dummy_sample   IgAKO_SH


In [26]:
sample_matched = groups_df.sample_name.isin(tree_data.columns)
print(sample_matched)


0      True
1      True
2      True
3      True
4      True
5      True
6      True
7      True
8      True
9      True
10    False
Name: sample_name, dtype: bool


In [34]:
if not sample_matched.all():
  unmatched = groups_df[~sample_matched].sample_name.tolist()
  print(unmatched)
  raise Exception("Check your data file and group file. Some samples in group file are not in data file column: " + ",".join(unmatched))


['dummy_sample']


Exception: Not all samples in group definition file in data columns:dummy_sample